In [1]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku 
import numpy as np

text = open('input.txt').read() # doom lyrics
print('corpus length:', len(text))

tokenizer = Tokenizer()

corpus length: 326073


In [2]:
def dataset_preparation(data):
    corpus = data.lower().split("\n")    
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1 
    
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences,   
                          maxlen=max_sequence_len, padding='pre'))
    
    predictors, label = input_sequences[:,:-1], input_sequences[:,-1]
    label = ku.to_categorical(label, num_classes=total_words)
    
    return predictors, label, max_sequence_len, total_words

In [3]:
def create_model(predictors, label, max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    
    model = Sequential()
    model.add(Embedding(total_words, 10, input_length=input_len))
    model.add(LSTM(150))
    model.add(Dropout(0.1))
    model.add(Dense(total_words, activation='softmax'))    
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    
    model.fit(predictors, label, epochs=100, verbose=1)
    
    return model

In [4]:
def generate_text(seed_text, next_words, max_sequence_len, model):
    for j in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen= 
                             max_sequence_len-1, padding='pre')
        predicted = model.predict_classes(token_list, verbose=0)
  
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

In [5]:
X, Y, max_len, total_words = dataset_preparation(text)
model = create_model(X, Y, max_len, total_words)

Epoch 1/100
1713/1713 [==============================] - 107s 62ms/step - loss: 7.2935
Epoch 2/100
1713/1713 [==============================] - 99s 58ms/step - loss: 6.9109
Epoch 3/100
1713/1713 [==============================] - 77s 45ms/step - loss: 6.7278
Epoch 4/100
1713/1713 [==============================] - 76s 44ms/step - loss: 6.5330
Epoch 5/100
1713/1713 [==============================] - 108s 63ms/step - loss: 6.3057
Epoch 6/100
1713/1713 [==============================] - 102s 59ms/step - loss: 6.0548
Epoch 7/100
1713/1713 [==============================] - 84s 49ms/step - loss: 5.7968
Epoch 8/100
1713/1713 [==============================] - 59s 35ms/step - loss: 5.5374 0s - loss: 5.
Epoch 9/100
1713/1713 [==============================] - 58s 34ms/step - loss: 5.2855
Epoch 10/100
1713/1713 [==============================] - 59s 34ms/step - loss: 5.0422
Epoch 11/100
1713/1713 [==============================] - 58s 34ms/step - loss: 4.8165
Epoch 12/100
1713/1713 [===========

1713/1713 [==============================] - 59s 35ms/step - loss: 1.5361
Epoch 96/100
1713/1713 [==============================] - 60s 35ms/step - loss: 1.4895
Epoch 97/100
1713/1713 [==============================] - 60s 35ms/step - loss: 1.4894
Epoch 98/100
1713/1713 [==============================] - 59s 35ms/step - loss: 1.5027
Epoch 99/100
1713/1713 [==============================] - 59s 34ms/step - loss: 1.4942
Epoch 100/100
1713/1713 [==============================] - 59s 34ms/step - loss: 1.4766
